$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
from delphes_reader import DelphesLoader #The class that allows us to access all paths of delphes files inside the server.
from delphes_reader import clasificator #It contains functions to classify particles.
from delphes_reader import root_analysis #It contains some useful functions like make_histograms or get_kinematics_row.
from delphes_reader import Quiet #Context manager for silencing certain ROOT operations.

from ROOT import TChain #It is necessary to read .root files.

import pandas as pd #Python library is useful for data science.

With the objective of watching Uniandes_Framework in action, we will reconstruct Z Boson Mass using some cuts and reading all background .root files. 

We are going to take as a starting point: **1_Read_Delphes_Files.ipynb**.

**1. Get delphes files (.root) paths.**

First, we must get delphes file paths of the signal. In this case, the signal is Z Boson. To do this, we can use DelphesLoader.

In [ ]:
Paths_Signal = DelphesLoader('Z_Tutorial').Forest
Paths_WW = DelphesLoader('ww').Forest
Paths_ttbar = DelphesLoader('ttbar').Forest
Paths_stop = DelphesLoader('stop').Forest

In [ ]:
Paths = {'z':Paths_Signal, 'ww': Paths_WW, 'ttbar': Paths_ttbar, 'stop': Paths_stop}

**2. Extract information from root files to create .CSV files:**

To do this, we have to read those .root files. 

If we want to adding cuts to the muons we have to do the following:

In [ ]:
#At first, we have to create a directory to save the z reconstructed kinematic information of 'z', 'ww', 'ttbar', 'stop' signals:
Z_reconstructed_particles = {}

with Quiet(): #Context manager for silencing certain ROOT operations.
    
    for key in Paths.keys(): #Paths.keys() is ['z', 'ww', 'ttbar', 'stop']
                
        for Path in Paths[key]: #Paths[key] is the list with the delphes file paths.
            
            #To read a .root file we have to create a Tree in ROOT and associate it the path.
            tree = TChain("Delphes;1") #Empty TChain of ROOT.
            tree.Add(Path) #Now we associate tree with the path of a .root file.

            #At this point we must go over all events in the tree, so:
            for event in tree:
                #Now we can use Uniandes_Framework to extract information:
                
                muons = clasificator.get_muons(event) #clasificator.get_muons(event) is a list that contains the muons presents in the event.
                
                if (len(muons) < 2): continue #The event is discarded if there are less than two muons
                                
                if(muons[0].GetCharge()*muons[1].GetCharge() > 0): continue #The event is discarded if they are not oppositely charged muons.

                if (muons[0].pt() < 30): continue #The event is discarded if muon[0] have not a pT > 30.
                
                if (muons[1].pt() < 20): continue #The event is discarded if muon[1] have not a pT > 30.
                
            break
        break
        

In fact, we can create a DataFrame to save the quantity of particles that surpass each cut. With this quantity we will be able to calculate the efficiency of each cut.

In [ ]:
#At first, we have to create a dictionary to save the z reconstructed kinematic information of 'z', 'ww', 'ttbar', 'stop' signals:
Z_reconstructed_particles = {}

#We also need a dictionary 
Cutflows = {}

with Quiet(): #Context manager for silencing certain ROOT operations.
    
    for key in Paths.keys(): #Paths.keys() is ['z', 'ww', 'ttbar', 'stop']
                
        for Path in Paths[key]: #Paths[key] is the list with the delphes file paths.
            
            #To read a .root file we have to create a Tree in ROOT and associate it the path.
            tree = TChain("Delphes;1") #Empty TChain of ROOT.
            tree.Add(Path) #Now we associate tree with the path of a .root file.

            #At this point we must go over all events in the tree, so:
            for event in tree:
                cut = 'All events'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                #Now we can use Uniandes_Framework to extract information:
                
                muons = clasificator.get_muons(event) #clasificator.get_muons(event) is a list that contains the muons presents in the event.
                
                if (len(muons) < 2): continue #The event is discarded if there are less than two muons
                cut = 'Events with least two muons'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                                
                if(muons[0].GetCharge()*muons[1].GetCharge() > 0): continue #The event is discarded if they are not oppositely charged muons.
                cut = 'Events with oppositely charged muons'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                if (muons[0].pt() < 30): continue #The event is discarded if muon[0] have not a pT > 30.
                cut = 'Events with PT(mu[0]) > 30'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                if (muons[1].pt() < 20): continue #The event is discarded if muon[1] have not a pT > 30.
                cut = 'Events with PT(mu[1]) < 20'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
            break
        break
        

In [ ]:
DataFrame_Cutflows = pd.DaFrame.from_dict(Cutflows)
DataFrame_Cutflows

Let's do this to create .csv files and cutflow file reading all .root files:

In [ ]:
#At first, we have to create a directory to save the z reconstructed kinematic information of 'z', 'ww', 'ttbar', 'stop' signals:
Z_reconstructed_particles = {}
Cutflows = {}

with Quiet(): #Context manager for silencing certain ROOT operations.
    
    for key in Paths.keys(): #Paths.keys() is ['z', 'ww', 'ttbar', 'stop']
        
        Z_reconstructed_particles[key] = [] #We have to create a list to save the kinematic information of z reconstructed for each key.
        
        for Path in Paths[key]: #Paths[key] is the list with the delphes file paths.
            
            #To read a .root file we have to create a Tree in ROOT and associate it the path.
            tree = TChain("Delphes;1") #Empty TChain of ROOT.
            tree.Add(Path) #Now we associate tree with the path of a .root file.

            #At this point we must go over all events in the tree, so:
            for event in tree:
                cut = 'All events'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                #Now we can use Uniandes_Framework to extract information:
                
                muons = clasificator.get_muons(event) #clasificator.get_muons(event) is a list that contains the muons presents in the event.
                
                if (len(muons) < 2): continue #The event is discarded if there are less than two muons
                cut = 'Events with least two muons'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                                
                if(muons[0].GetCharge()*muons[1].GetCharge() > 0): continue #The event is discarded if they are not oppositely charged muons.
                cut = 'Events with oppositely charged muons'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                if (muons[0].pt() < 30): continue #The event is discarded if muon[0] have not a pT > 30.
                cut = 'Events with PT(mu[0]) > 30 (GeV)'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                if (muons[1].pt() < 20): continue #The event is discarded if muon[1] have not a pT > 30.
                cut = 'Events with PT(mu[1]) < 20 (GeV)'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                if(abs(muons[0].eta()) > 2.4 or abs(muons[1].eta()) > 2.4): continue
                cut = '|Eta| < 2.4 '
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                if(muons[0].DeltaR(muons[1]) < 0.3): continue
                cut = 'DeltaR > 0.3'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                #How we can reconstruct Z Boson? We have to sum the TLorentz vector of the muon pair: 
                Z = muons[0].GetTLV() + muons[1].GetTLV()
                
                if(Z.M() < 66): continue
                if(Z.M() > 116): continue
                cut = '66 (GeV) < M(Z) < 116 (Gev)'
                Cutflows[cut] = Cutflows.get(cut,0) + 1
                
                #However, Z is not an object of particle class, it is a TLV. So, we can not use root_analysis.get_kinematics_row().  
                #If we want his kinematic information we have to create the directory by ourselves:
                Z_kinematic_information = {"pT_{Z}(GeV)": Z.Pt(), 
                                           "#eta_{Z}": Z.Eta(), 
                                           "#phi_{Z}": Z.Phi(), 
                                           "Energy_{Z}(GeV)": Z.Energy(), 
                                           "Mass_{Z}(GeV)" : Z.M()} 
                
                #This directory has the same structure of an get_kinematics_row() output. 
                
                #Finally, we can save Z_kinematic_information for each event in a list:
                Z_reconstructed_particles[key].append(Z_kinematic_information)  
            break
        break
        
        #At this point we have Z_reconstructed_particles[key] (a list) filled with kinematic information, so we must save it:
        root_analysis.generate_csv(Z_reconstructed_particles[key], f'Data_{key}.csv')
        DataFrame_Cutflows = pd.DaFrame.from_dict(Cutflows)
        DataFrame_Cutflows.to_csv('Cutflows.csv')

Until now, we created .csv files. Now we will use it to plot histograms. For this we are going to take as a starting point: **2_Histograms.ipynb**.

**3. Read .csv files:**

To do this, we can use pandas:

In [ ]:
Datasets = {'z': pd.read_csv('Data_z.csv'),
            'stop': pd.read_csv('Data_stop.csv'),
            'ttbar': pd.read_csv('Data_ttbar.csv'),
            'ww': pd.read_csv('Data_ww.csv')}

**4. Create histograms:**

We can use the function **root_analysis.makehistograms()**. If we want to generate all the histograms for each column on Datasets['stop'], it is enough to run the following code:

In [ ]:
Histos_Dictionary = {'z': root_analysis.make_histograms(Datasets['z']),
                    'stop': root_analysis.make_histograms(Datasets['stop']),
                    'ttbar': root_analysis.make_histograms(Datasets['ttbar']),
                    'ww': root_analysis.make_histograms(Datasets['ww'])}

**5. Overlap histograms:**

We can use the function **root_analysis.overlap_histos()**. This function has two main parameters: kinematic_variable and Dict_Histos, they are a string with the name of the kinematic variable and the directory with all the histograms (it should be a directory with the same structure that we use in this tutorial) respectively.

In [ ]:
Histos, Canva, Legend = root_analysis.overlap_histos(kinematic_variable= 'Mass_{Z}(GeV)', 
                                                     Dict_Histos= Histos_Dictionary)

**6. Stack histograms:**

For this we can use the function **root_analysis.overlap_histos()** and setting its parameter "Stack" as True (Boolean). However, before using Stack we have to normalized all histograms with the number of physical events. This is the reason about why we created Cutflows file.

In order to calcule the number of physical events we can use the following function:

In [ ]:
def N_events(signal):
    cutflows = pd.read_csv('Cutflows.csv')
    
    Efficience = cutflows[signal][-1]/cutflows[signal][0]
    xs = DelphesLoader(signal).xs
    Luminosity = 10*1000
    
    return Efficience*Luminosity*xs

Let's normalizate all histograms using this function:

In [ ]:
for signal in Histos_Dictionary.keys():
    N = N_events(signal)
    for histo in Histos_Dictionary[signal]:
        histo.scale(N/histo.Integral())

Right now, we can stack the histograms without any problem:

In [ ]:
Histos, Canva, Legend = root_analysis.overlap_histos(kinematic_variable= 'Mass_{Z}(GeV)', 
                                                     Dict_Histos= Histos_Dictionary,
                                                     Stack= True)

This agrees with plots reported in: https://cds.cern.ch/record/2707171?ln=es